# NoSQL (MongoDB) (sesión 3)

![Image of MongoDB](https://webassets.mongodb.com/_com_assets/cms/MongoDB-Logo-5c3a7405a85675366beb3a5ec4c032348c390b3f142f5e6dddf1d78e2df5cb5c.png)

Esta hoja muestra cómo acceder a bases de datos MongoDB y también a conectar la salida con Jupyter. Se puede utilizar el *shell* propio de MongoDB en la máquina virtual usando el programa `mongo`. La diferencia es que ese programa espera código Javascript y aquí trabajaremos con Python.

Lo primero que se hará será instalar la versión 3.2 de mongodb. Ésta no viene por defecto en la máquina virtual, y sólo hay que hacerlo una vez, aunque no pasa nada si se repite.

In [ ]:
%%bash
sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv EA312927
echo "deb http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.2 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-3.2.list
sudo apt-get update
sudo apt-get install -y mongodb-org

In [ ]:
!sudo service mongod start

In [ ]:
!mongo -version

In [ ]:
!echo 'show log global' | mongo

In [ ]:
from pprint import pprint as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline
matplotlib.style.use('ggplot')

Usaremos la librería `pymongo` para python. La cargamos a continuación.

In [ ]:
import pymongo
from pymongo import MongoClient

La conexión se inicia con `MongoClient`.

In [ ]:
client = MongoClient("localhost",27017)
client

In [ ]:
client.database_names()

 - Format: 7zipped
 - Files:
   - **badges**.xml
       - UserId, e.g.: "420"
       - Name, e.g.: "Teacher"
       - Date, e.g.: "2008-09-15T08:55:03.923"
   - **comments**.xml
       - Id
       - PostId
       - Score
       - Text, e.g.: "@Stu Thompson: Seems possible to me - why not try it?"
       - CreationDate, e.g.:"2008-09-06T08:07:10.730"
       - UserId
   - **posts**.xml
       - Id
       - PostTypeId
          - 1: Question
          - 2: Answer
       - ParentID (only present if PostTypeId is 2)
       - AcceptedAnswerId (only present if PostTypeId is 1)
       - CreationDate
       - Score
       - ViewCount
       - Body
       - OwnerUserId
       - LastEditorUserId
       - LastEditorDisplayName="Jeff Atwood"
       - LastEditDate="2009-03-05T22:28:34.823"
       - LastActivityDate="2009-03-11T12:51:01.480"
       - CommunityOwnedDate="2009-03-11T12:51:01.480"
       - ClosedDate="2009-03-11T12:51:01.480"
       - Title=
       - Tags=
       - AnswerCount
       - CommentCount
       - FavoriteCount
   - **posthistory**.xml
	   - Id
	   - PostHistoryTypeId
			- 1: Initial Title - The first title a question is asked with.
			- 2: Initial Body - The first raw body text a post is submitted with.
			- 3: Initial Tags - The first tags a question is asked with.
			- 4: Edit Title - A question's title has been changed.
			- 5: Edit Body - A post's body has been changed, the raw text is stored here as markdown.
			- 6: Edit Tags - A question's tags have been changed.
			- 7: Rollback Title - A question's title has reverted to a previous version.
			- 8: Rollback Body - A post's body has reverted to a previous version - the raw text is stored here.
			- 9: Rollback Tags - A question's tags have reverted to a previous version.
			- 10: Post Closed - A post was voted to be closed.
			- 11: Post Reopened - A post was voted to be reopened.
			- 12: Post Deleted - A post was voted to be removed.
			- 13: Post Undeleted - A post was voted to be restored.
			- 14: Post Locked - A post was locked by a moderator.
			- 15: Post Unlocked - A post was unlocked by a moderator.
			- 16: Community Owned - A post has become community owned.
			- 17: Post Migrated - A post was migrated.
			- 18: Question Merged - A question has had another, deleted question merged into itself.
			- 19: Question Protected - A question was protected by a moderator
			- 20: Question Unprotected - A question was unprotected by a moderator
			- 21: Post Disassociated - An admin removes the OwnerUserId from a post.
			- 22: Question Unmerged - A previously merged question has had its answers and votes restored.
		- PostId
		- RevisionGUID: At times more than one type of history record can be recorded by a single action.  All of these will be grouped using the same RevisionGUID
		- CreationDate: "2009-03-05T22:28:34.823"
		- UserId
		- UserDisplayName: populated if a user has been removed and no longer referenced by user Id
		- Comment: This field will contain the comment made by the user who edited a post
		- Text: A raw version of the new value for a given revision
			- If PostHistoryTypeId = 10, 11, 12, 13, 14, or 15  this column will contain a JSON encoded string with all users who have voted for the PostHistoryTypeId
			- If PostHistoryTypeId = 17 this column will contain migration details of either "from <url>" or "to <url>"
		- CloseReasonId
			- 1: Exact Duplicate - This question covers exactly the same ground as earlier questions on this topic; its answers may be merged with another identical question.
			- 2: off-topic
			- 3: subjective
			- 4: not a real question
			- 7: too localized
   - **postlinks**.xml
     - Id
     - CreationDate
     - PostId
     - RelatedPostId
     - PostLinkTypeId
       - 1: Linked
       - 3: Duplicate
   - **users**.xml
     - Id
     - Reputation
     - CreationDate
     - DisplayName
     - EmailHash
     - LastAccessDate
     - WebsiteUrl
     - Location
     - Age
     - AboutMe
     - Views
     - UpVotes
     - DownVotes
   - **votes**.xml
     - Id
     - PostId
     - VoteTypeId
        - ` 1`: AcceptedByOriginator
        - ` 2`: UpMod
        - ` 3`: DownMod
        - ` 4`: Offensive
        - ` 5`: Favorite - if VoteTypeId = 5 UserId will be populated
        - ` 6`: Close
        - ` 7`: Reopen
        - ` 8`: BountyStart
        - ` 9`: BountyClose
        - `10`: Deletion
        - `11`: Undeletion
        - `12`: Spam
        - `13`: InformModerator
     - CreationDate
     - UserId (only for VoteTypeId 5)
     - BountyAmount (only for VoteTypeId 9)

In [ ]:
%%bash
(test -e /vagrant/Posts.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Posts.csv.gz -O - 2>/dev/null | gunzip > /vagrant/Posts.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e /vagrant/Users.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Users.csv.gz -O - 2>/dev/null | gunzip > /vagrant/Users.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e /vagrant/Tags.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Tags.csv.gz -O - 2>/dev/null | gunzip > /vagrant/Tags.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e /vagrant/Comments.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Comments.csv.gz -O - 2>/dev/null | gunzip > /vagrant/Comments.csv) \
  && echo OK)

In [ ]:
%%bash
(test -e /vagrant/Votes.csv && echo "Ya descargado") || (\
(wget http://neuromancer.inf.um.es:8080/es.stackoverflow/Votes.csv.gz -O - 2>/dev/null | gunzip > /vagrant/Votes.csv) \
  && echo OK)

Las bases de datos se crean conforme se nombran. Se puede utilizar la notación punto o la de diccionario. Las colecciones también.

In [ ]:
db = client.stackoverflow
db = client['stackoverflow']
db

Las bases de datos están compuestas por un conjunto de **colecciones**. Cada colección aglutina a un conjunto de objetos (documentos) del mismo tipo, aunque como vimos en teoría, cada documento puede tener un conjunto de atributos diferente.

In [ ]:
posts = db.posts
posts

Importación de los ficheros CSV. Por ahora creamos una colección diferente para cada uno. Después estudiaremos cómo poder optimizar el acceso usando agregación.

In [ ]:
%%bash
mongoimport --db stackoverflow --collection posts --drop --type csv \
   --headerline --host=127.0.0.1 --file /vagrant/Posts.csv

In [ ]:
%%bash
mongoimport --db stackoverflow --collection users --drop --type csv \
   --headerline --host=127.0.0.1 --file /vagrant/Users.csv

In [ ]:
%%bash
mongoimport --db stackoverflow --collection votes --drop --type csv \
   --headerline --host=127.0.0.1 --file /vagrant/Votes.csv

In [ ]:
%%bash
mongoimport --db stackoverflow --collection comments --drop --type csv \
   --headerline --host=127.0.0.1 --file /vagrant/Comments.csv

In [ ]:
%%bash
mongoimport --db stackoverflow --collection tags --drop --type csv \
   --headerline --host=127.0.0.1 --file /vagrant/Tags.csv

In [ ]:
posts.count()

El API de colección en Python se puede encontrar aquí: https://api.mongodb.com/python/current/api/pymongo/collection.html. La mayoría de libros y referencias muestran el uso de mongo desde Javascript, ya que el *shell* de MongoDB acepta ese lenguaje. La sintaxis con respecto a Python cambia un poco, y se puede seguir en el enlace anterior.

In [ ]:
post = posts.find_one()
post

Utilizo la librería `pp` para imprimir los objetos grandes de una manera amigable.

In [ ]:
users = db.users
pp(users.find_one())

A cada objeto se le asigna una clave implícita con nombre "`_id`" (si el objeto no lo incluye). Además, se pueden crear más índices, de tipos `ASCENDING`, `DESCENDING`, `HASHED`, y otros geoespaciales. https://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.create_index

In [ ]:
print type(post['_id'])
post['_id']

In [ ]:
print post['_id'].generation_time.strftime("%Y-%m-%d %H:%M:%SZ%z")

In [ ]:
from datetime import datetime
iso_date = datetime.strptime(post['CreationDate'], "%Y-%m-%dT%H:%M:%S.%f")
post['CreationDate'] = iso_date
post

La siguiente sintaxis está descatalogada en las nuevas versiones, pero era más conveniente:

In [ ]:
posts.save(post)

Ahora hay que hacerlo así (el resultado debe ser 1 documento modificado):

In [ ]:
result = posts.replace_one({"_id": post['_id']}, post)
result.modified_count

In [ ]:
post = posts.find_one()
post

(Al menos para el notebook, imprimiendo los campos se ve bien las tildes en Unicode).

In [ ]:
for k,v in post.iteritems():
    print "%s: %s" % (k,v)

Además de `find_one()`, la función principal de búsqueda es `find()`. Esta función ofrece un conjunto muy ámplio de opciones para búsqueda, que estudiaremos a continuación.

Primero, una consulta sencilla, con el valor de un campo:

In [ ]:
respuestas = posts.find({"PostTypeId": 2})
respuestas.count()

También existe `explain()`, al estilo de SQL.

In [ ]:
posts.find({"PostTypeId": 2}).explain()

También se puede limitar la búsqueda.

In [ ]:
respuestas = posts.find({"PostTypeId": 2}).limit(10)

La respuesta no es un conjunto de elementos, sino un cursor que puede ir recorriéndose.

In [ ]:
respuestas

In [ ]:
def print_respuestas(respuestas):
    for r in respuestas:
        print pp(r)
        
print_respuestas(respuestas)

También se puede importar en un dataframe de pandas:

In [ ]:
respuestas = posts.find({"PostTypeId": 2}).limit(10)
df = pd.DataFrame([r['Id'] for r in respuestas])
df.plot()

La función `find()` tiene un gran número de posibilidades para especificar la búsqueda. Se pueden utilizar cualificadores complejos como:

- `$and`
- `$or`
- `$not`

Estos calificadores unen "objetos", no valores. Por otro lado, hay otros calificadores que se refieren a valores:

- `$lt` (menor)
- `$lte` (menor o igual)
- `$gt` (mayor)
- `$gte` (mayor o igual)

In [ ]:
respuestas = posts.find({ '$and' : [{"PostTypeId": 2} ,
                                    {"Id" : {'$gte' : 100}}]}).limit(10)
print_respuestas(respuestas)

Framework de agregación: https://docs.mongodb.com/manual/reference/operator/aggregation/. Y aquí una presentación interesante sobre el tema: https://www.mongodb.com/presentations/aggregation-framework-0?jmp=docs&_ga=1.223708571.1466850754.1477658152

In [ ]:
respuestas = posts.aggregate( [ {'$project' : { 'Id' : 1 }}, {'$limit': 20} ])
print_respuestas(respuestas)

In [ ]:
posts.create_index([('Id', pymongo.HASHED)])

## Esquemas: Monothon

El siguiente comando instala la librería python `monothon`, que no viene instalada en la máquina virtual. Esta librería nos permite definir esquemas por los que se regirán los datos de las colecciones. Está basada en la librería para Javascript `mongoose`.

In [ ]:
!sudo pip install mongothon